insert table of contents here
# Introduction


In [1]:
import sys

In [134]:
import sys
import os

# mainJplotReader.py 파일이 있는 경로를 추가
module_path ='C:\Program Files\JMAG-Designer23.1\Tools\JPlotReader\samples\Python'
if module_path not in sys.path:
    sys.path.append(module_path)

# 모듈 가져오기
import mainJplotReader as jpr

In [27]:
import ctypes
import scipy.io as sio
from JplotReader import JplotReader as jr
import os
import re

In [ ]:
base_jplot_path = r"D:\KangDH\Thesis\e10\JMAG\MSConductorModel\e10MS_ConductorModel_REF.jfiles\e10MS_ConductorModel_REF~7\e10MS_ConductorModel_REF_Load~16\Case{}\Designer.jplot"
SCL_jplot_path = r"D:\KangDH\Thesis\e10\JMAG\MSConductorModel\e10MS_ConductorModel.jfiles\e10MS_ConductorModel~6\e10MS_ConductorModel_SCL_Load~13\Case{}\Designer.jplot"
# Case1부터 Case30까지 처리
for case_number in range(1, 31):
    # 각 case 번호에 따라 경로 생성
    current_jplot_path = SCL_jplot_path.format(case_number)
    postkey_id = 16001
    jpr.main(current_jplot_path, 121,postkey_id)


In [135]:
# base_jplot_path = r"D:\KangDH\Thesis\e10\JMAG\MSConductorModel\e10MS_ConductorModel_REF.jfiles\e10MS_ConductorModel_REF~7\e10MS_ConductorModel_REF_Load~16\Case{}\Designer.jplot"
base_jplot_path=r"D:\KangDH\Thesis\e10\JMAG\REF_e10_WTPM_PatternD_R1_18k.jfiles\ref_e10_WirePeriodic~1\ref_e10_WirePeriodic_Load_18k_rgh~32\Case{}\Designer.jplot"
# Case1부터 Case30까지 처리
# for case_number in range(1, 31):
    # 각 case 번호에 따라 경로 생성
current_jplot_path = base_jplot_path.format(28)
postkey_id = 16001
current_jplot_path
jpr.main(current_jplot_path, 241,postkey_id)

Data saved to ref_e10_WirePeriodic_Load_18k_rgh~32_Case28_MagB.mat


# WIP Part별 그룹

In [128]:
import sys
import os
import ctypes
import scipy.io as sio
import re

# mainJplotReader.py 파일이 있는 경로를 추가
module_path = r'C:\Program Files\JMAG-Designer23.1\Tools\JPlotReader\samples\Python'
if module_path not in sys.path:
    sys.path.append(module_path)

# 모듈 가져오기
import mainJplotReader as jpr

# savePartGroupedDataToMat 함수: 파트별로 데이터를 그룹핑하여 mat 파일로 저장
def savePartGroupedDataToMat(part_grouped_data, output_file_name):
    """
    파트별로 그룹핑된 데이터를 .mat 파일로 저장하는 함수.
    
    Args:
        part_grouped_data (dict): 파트 이름별로 그룹핑된 데이터.
        output_file_name (str): 저장할 mat 파일 이름.
    """
    mat_data = {}
    
    # 파트 그룹핑된 데이터를 mat 파일 형식에 맞게 저장
    for part_name, data in part_grouped_data.items():
        # 파트 이름의 특수문자 제거 및 길이 제한
        valid_part_name = re.sub(r'\W+', '_', part_name)
        mat_data[valid_part_name] = data
    
    # .mat 파일로 저장
    sio.savemat(output_file_name, mat_data)
    print(f"파트별 데이터가 {output_file_name}으로 저장되었습니다.")

# main 함수 수정
def main(jplot_file_path, totalSteps, postkey_id):
    # JplotReader 열기
    reader = jpr.JplotReader.jplotOpen(jplot_file_path.encode('utf-8'))

    if reader is None:
        print(f"Cannot open file: {jplot_file_path}")
        return
    
    part_grouped_data = {}
    
    # 파트 정보 읽기 (파트 ID와 이름을 가져와서 데이터 그룹핑에 사용)
    numParts = jpr.JplotReader.jplotCountParts(reader)
    part_info = {}
    for i in range(numParts):
        part_id = ctypes.c_int()
        name_buffer = ctypes.create_string_buffer(256)  # 파트 이름을 저장할 버퍼
        jpr.JplotReader.jplotGetPartIdTitle(reader, i, ctypes.byref(part_id), name_buffer, 256)
        part_name = name_buffer.value.decode('utf-8')
        part_info[part_id.value] = part_name
    
    # 요소 중심 읽기
    element_centers = jpr.readElementCenters(reader)

    # 각 스텝에 대한 컴포넌트 데이터 읽기 및 파트 이름에 따라 그룹핑
    for stepIndex in range(1, totalSteps + 1):
        component_data = jpr.readComponentData(reader, postkey_id, stepIndex)
        
        for element in component_data:
            element_id, value1, value2, value3 = element
            part_id = None
            
            # 요소 ID를 통해 part_id 찾기
            for center in element_centers:
                if center[0] == element_id:  # center[0]은 element_id
                    part_id = center[1]  # center[1]은 part_id
                    break
            
            # 파트 이름으로 데이터 그룹핑
            if part_id in part_info:
                part_name = part_info[part_id]
                if part_name not in part_grouped_data:
                    part_grouped_data[part_name] = []
                part_grouped_data[part_name].append([element_id, value1, value2, value3])
    
    # JplotReader 닫기
    jpr.JplotReader.jplotClose(reader)

    # 파일 경로에서 Case 번호 및 로드 이름 추출
    case_match = re.search(r"\\(Case\d+)", jplot_file_path)
    case_name = case_match.group(1)  # 'Case1', 'Case2' 등의 Case 번호 추출

    load_name_match = re.search(r"\\([^\\]+)\\Case\d+", jplot_file_path)
    if load_name_match:
        load_name = load_name_match.group(1)  # "Case" 이전의 폴더 이름
    else:
        load_name = "UnknownLoad"  # 추출 실패 시 기본값

    # 파일 이름 생성
    if postkey_id == 16001:
         output_file_name = f"{load_name}_{case_name}_MagB_PartGrouped.mat"
    elif postkey_id == 11005:
        output_file_name = f"{load_name}_{case_name}_MagA_PartGrouped.mat"

    # 그룹핑된 데이터를 .mat 파일로 저장
    savePartGroupedDataToMat(part_grouped_data, output_file_name)

    print(f"Data saved to {output_file_name}")

In [133]:
# base_jplot_path = r"D:\KangDH\Thesis\e10\JMAG\MSConductorModel\e10MS_ConductorModel_REF.jfiles\e10MS_ConductorModel_REF~7\e10MS_ConductorModel_REF_Load~16\Case{}\Designer.jplot"
base_jplot_path=r"D:\KangDH\Thesis\e10\JMAG\REF_e10_WTPM_PatternD_R1_18k.jfiles\ref_e10_WirePeriodic~1\ref_e10_WirePeriodic_Load_18k_rgh~32\Case{}\Designer.jplot"
# Case1부터 Case30까지 처리
# for case_number in range(1, 31):
    # 각 case 번호에 따라 경로 생성
current_jplot_path = base_jplot_path.format(28)
postkey_id = 16001
current_jplot_path
main(current_jplot_path, 241,postkey_id)

파트별 데이터가 ref_e10_WirePeriodic_Load_18k_rgh~32_Case28_MagB_PartGrouped.mat으로 저장되었습니다.
Data saved to ref_e10_WirePeriodic_Load_18k_rgh~32_Case28_MagB_PartGrouped.mat


# Manual JPlotReader Call

In [47]:

import ctypes
import sys

from JplotReader import JplotReader

def main():
    if len(sys.argv) != 2:
        print("Parameter should be path to JPLOT file")
        sys.exit(1)

    # Open JplotReader for the given path
    reader = JplotReader.jplotOpen(sys.argv[1].encode('utf-8'))

    if reader is None:
        print("Cannot open file: {:s}".format(sys.argv[1]))
        sys.exit(1)
    
    # Read nodes
    readNodes(reader)

    # Read element centers
    readElementCenters(reader)

    # Read nodal displacements
    readNodalDisplacements(reader)

    # Read element center displacements
    readElementCenterDisplacements(reader)

    # Read component data
    readComponentData(reader)

    # Close JplotReader when finished
    JplotReader.jplotClose(reader)

    sys.exit(0)

def readNodes(reader):
    nodeCount = JplotReader.jplotCountNodes(reader) # Number of nodes
    print ("Number of nodes: {:d}".format(nodeCount))

    JplotReader.jplotStartPopNode(reader) # Prepare reader to pop nodes
    for i in range(nodeCount):
        id = ctypes.c_int() # Node ID
        x = ctypes.c_double() # Coordinates of node
        y = ctypes.c_double()
        z = ctypes.c_double()
        JplotReader.jplotPopNode(reader, id, x, y, z) # Pop node off stack and get its data
        print("Id: {:d}, x: {:f}, y: {:f}, z: {:f}".format(id.value, x.value, y.value, z.value))

    JplotReader.jplotEndPopNode(reader) # End popping of nodes

def readElementCenters(reader):
    elementCount = JplotReader.jplotCountElements(reader) # Number of elements
    print ("Number of elements: {:d}".format(elementCount))

    JplotReader.jplotStartPopElementCenter(reader) # Prepare reader to pop elements
    for i in range(elementCount):
        id = ctypes.c_int() # Element ID
        partId = ctypes.c_int() # Material ID
        eleType = ctypes.c_int() # Shape type of element
        x = ctypes.c_double() # Coordinates of center of element
        y = ctypes.c_double()
        z = ctypes.c_double()
        area = ctypes.c_double() # Area/volume of element
        JplotReader.jplotPopElementCenter(reader, id, partId, eleType, x, y, z, area) # Pop element off stack and get its data
        print("Id: {:d}, Part Id: {:d}, Element type: {:d}, x: {:f}, y: {:f}, z: {:f}, area: {:f}".format(id.value, partId.value, eleType.value, x.value, y.value, z.value, area.value))

    JplotReader.jplotEndPopElementCenter(reader) # End popping of elements

def readNodalDisplacements(reader):
    step = 1 # Step to read
    displacementCount = JplotReader.jplotCountNodalDisplacements(reader, step) # Number of nodal displacements
    print("Number of nodal displacements: {:d}".format(displacementCount))

    JplotReader.jplotStartPopNodalDisplacement(reader, step) # Prepare reader to pop nodal displacements
    for i in range(displacementCount):
        id = ctypes.c_int() # Node ID
        dx = ctypes.c_double() # Change in node position (displacement)
        dy = ctypes.c_double()
        dz = ctypes.c_double()
        JplotReader.jplotPopNodalDisplacement(reader, id, dx, dy, dz) # Pop nodal displacement off stack and get its data
        print("Id: {:d}, dx: {:f}, dy: {:f}, dz: {:f}".format(id.value, dx.value, dy.value, dz.value))

    JplotReader.jplotEndPopNodalDisplacement(reader) # End popping of nodal displacements

def readElementCenterDisplacements(reader):
    step = 1 # Step to read
    displacementCount = JplotReader.jplotCountElementCenterDisplacements(reader, step) # Number of element center displacements
    print("Number of element center displacements: {:d}".format(displacementCount))

    JplotReader.jplotStartPopElementCenterDisplacements(reader, step) # Prepare reader to pop element center displacements
    for i in range(displacementCount):
        id = ctypes.c_int() # Element ID
        dx = ctypes.c_double() # Change in element center position (displacement)
        dy = ctypes.c_double()
        dz = ctypes.c_double()
        JplotReader.jplotPopElementCenterDisplacement(reader, id, dx, dy, dz) # Pop element off stack and get its data
        print("Id: {:d}, dx: {:f}, dy: {:f}, dz: {:f}".format(id.value, dx.value, dy.value, dz.value))

    JplotReader.jplotEndPopElementCenterDisplacement(reader) # End popping of element center displacements

def readComponentData(reader):
    step = 1 # Step to read
    magneticFluxDensity = 16001 # Component key to read

    componentReader = JplotReader.jplotCreateComponentReader(reader, step, magneticFluxDensity) # Create component reader object
    if componentReader is None: # Check that component reader was properly created with valid values
        return
    
    componentCount = JplotReader.jplotCountComponents(componentReader) # Number of component data points
    print("Number of components: {:d}".format(componentCount))

    JplotReader.jplotStartPopComponent(componentReader) # Prepare component reader to pop component data
    for i in range(componentCount):
        id = ctypes.c_int()
        values = (ctypes.c_double * 3)() # Component values
        JplotReader.jplotPopComponent(componentReader, id, values) # Pop element off stack and get component data
        print("Id: {:d}, {:f}, {:f}, {:f}".format(id.value, values[0], values[1], values[2]))
    
    JplotReader.jplotEndPopComponent(componentReader) # End popping of component data
    JplotReader.jplotDeleteComponentReader(componentReader) # Delete component reader when done using


## Open Reader n getPartID

In [28]:
reader = jr.jplotOpen(current_jplot_path.encode('utf-8'))

In [50]:
NumPart=jr.jplotCountParts(reader)

In [57]:

# 파트 ID와 이름을 저장할 변수 정의
part_id = ctypes.c_int()  # 파트 ID 저장
name_buffer_size = 256    # 파트 이름을 저장할 버퍼 크기
name_buffer = ctypes.create_string_buffer(name_buffer_size)  # 파트 이름 저장을 위한 버퍼

# 파트 인덱스 설정 (예시로 0 사용)
part_index = 1

# jplotGetPartIdTitle 함수 호출
jr.jplotGetPartIdTitle(reader, part_index, ctypes.byref(part_id), name_buffer, name_buffer_size)

In [61]:
# 파트 목록 저장 리스트
part_name_list = []

# part_index를 증가시키며 반복적으로 조회
part_index = 0

while True:
    # 파트 ID 저장을 위한 변수
    part_id = ctypes.c_int()

    # jplotGetPartIdTitle 함수 호출
    jr.jplotGetPartIdTitle(reader, part_index, ctypes.byref(part_id), name_buffer, name_buffer_size)

    # name_buffer에 저장된 값 확인
    part_name = name_buffer.value.decode('utf-8')

    # part_name이 비어 있으면 더 이상 유효한 파트가 없다는 의미
    if part_name == '':
        break

    # 파트 이름 목록에 추가
    part_name_list.append(part_name.strip())

    # 다음 파트를 조회하기 위해 part_index 증가
    part_index += 1

# 결과 출력
print("Part Name List:")
for i, name in enumerate(part_name_list):
    print(f"Part {i+1}: {name}")


Part Name List:
Part 1: Rotor/airRegion.2@Air
Part 2: Rotor/airRegion.2@Air
Part 3: Rotor/Magnet2@N42EH
Part 4: Rotor/Magnet2@N42EH
Part 5: Rotor/airRegion@Air
Part 6: Rotor/airRegion@Air
Part 7: Rotor/Magnet1@N42EH
Part 8: Rotor/Magnet1@N42EH
Part 9: Rotor/RotorCore@NO18-1160
Part 10: Rotor/Shaft@Air
Part 11: Stator/Slot6/Wire1@Copper
Part 12: Stator/Slot4/Wire1@Copper
Part 13: Stator/Slot2/Wire1@Copper
Part 14: Stator/Slot5/Wire2@Copper
Part 15: Stator/Slot3/Wire2@Copper
Part 16: Stator/Slot6/Wire3@Copper
Part 17: Stator/Slot4/Wire3@Copper
Part 18: Stator/Slot2/Wire3@Copper
Part 19: Stator/Slot5/Wire4@Copper
Part 20: Stator/Slot3/Wire4@Copper
Part 21: Stator/Slot2/Wire4@Copper
Part 22: Stator/Slot4/Wire4@Copper
Part 23: Stator/Slot6/Wire4@Copper
Part 24: Stator/Slot3/Wire3@Copper
Part 25: Stator/Slot5/Wire3@Copper
Part 26: Stator/Slot2/Wire2@Copper
Part 27: Stator/Slot4/Wire2@Copper
Part 28: Stator/Slot6/Wire2@Copper
Part 29: Stator/Slot3/Wire1@Copper
Part 30: Stator/Slot5/Wire1@Copp

In [ ]:
readComponentData

In [49]:
readElementCenters(reader)

Number of elements: 27274
Id: 1, Part Id: 12, Element type: 2, x: 0.146853, y: 0.032805, z: 0.000000, area: 0.000000
Id: 2, Part Id: 14, Element type: 2, x: 0.147414, y: 0.012781, z: 0.000000, area: 0.000000
Id: 3, Part Id: 12, Element type: 2, x: 0.139613, y: 0.056394, z: 0.000000, area: 0.000001
Id: 4, Part Id: 12, Element type: 2, x: 0.138567, y: 0.056354, z: 0.000000, area: 0.000001
Id: 5, Part Id: 12, Element type: 2, x: 0.137429, y: 0.055314, z: 0.000000, area: 0.000001
Id: 6, Part Id: 12, Element type: 2, x: 0.138236, y: 0.055736, z: 0.000000, area: 0.000001
Id: 7, Part Id: 12, Element type: 2, x: 0.145653, y: 0.035317, z: 0.000000, area: 0.000000
Id: 8, Part Id: 12, Element type: 2, x: 0.145273, y: 0.035827, z: 0.000000, area: 0.000000
Id: 9, Part Id: 12, Element type: 2, x: 0.144781, y: 0.036985, z: 0.000000, area: 0.000000
Id: 10, Part Id: 12, Element type: 2, x: 0.144401, y: 0.037495, z: 0.000000, area: 0.000000
Id: 11, Part Id: 12, Element type: 2, x: 0.143908, y: 0.038654,

# J calc From MVP

In [ ]:

JplotReader.main(jplot_file_path,121)

In [ ]:
import ctypes
import scipy.io as sio
from JplotReader import JplotReader
import os
import re

def main(jplot_file_path, totalSteps):
    # Open JplotReader for the given path
    reader = JplotReader.jplotOpen(jplot_file_path.encode('utf-8'))

    if reader is None:
        print(f"Cannot open file: {jplot_file_path}")
        return
    
    data = {}
    
    # Read nodes
    data['nodes'] = readNodes(reader)

    # Read element centers
    data['element_centers'] = readElementCenters(reader)

    # Read nodal displacements
    data['nodal_displacements'] = readNodalDisplacements(reader)

    # Read element center displacements
    data['element_center_displacements'] = readElementCenterDisplacements(reader)
    
    # Read component data for each step
    for stepIndex in range(1, totalSteps+1):
        data[f'MagB_{stepIndex}'] = readComponentData(reader, 16001, stepIndex)
        data[f'MagA_{stepIndex}'] = readComponentData(reader, 11005, stepIndex)

    # Close JplotReader when finished
    JplotReader.jplotClose(reader)

    # Extract Case number and load name from the file path
    case_match = re.search(r"\\(Case\d+)", jplot_file_path)
    case_name = case_match.group(1)  # Extract the 'Case1', 'Case2', etc.

    # Extract the folder before "Case" to get the load name
    load_name_match = re.search(r"\\([^\\]+)\\Case\d+", jplot_file_path)
    if load_name_match:
        load_name = load_name_match.group(1)  # Folder name just before "Case"
    else:
        load_name = "UnknownLoad"  # Default name if extraction fails

    # Create the output file name using the extracted parts
    output_file_name = f"{load_name}_{case_name}.mat"

    # Save all data to .mat file with the new name
    sio.savemat(output_file_name, data)

    print(f"Data saved to {output_file_name}")

# Export MQS B 